In [1]:
import pandas as pd

from time import sleep
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException



In [17]:
 def getComments(url):

    # Set up Chrome driver options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    # Initialize ChromeDriver with an implicit wait of 20 seconds
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(20)

    # Open the video page
    driver.get(url)
    sleep(7)

    # Wait for the comment section to load
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="continuations"]')))

    scroll_pause_time = 10
    scroll_count = 0
    comment_list = []

    while len(comment_list) < 50 and scroll_count < 10:
        # Scroll down to the bottom of the page to load more comments
        driver.execute_script('window.scrollTo(0, document.documentElement.scrollHeight);')
        scroll_count += 1
        sleep(scroll_pause_time)

        # Find the comment div element and extract the comments
        comment_div = driver.find_element(By.XPATH, '//*[@id="contents"]')
        comments = comment_div.find_elements(By.XPATH, '//*[@id="content-text"]')
        for comment in comments:
            comment_list.append(comment.text.strip())
            
    # print(f"Scraped {len(comment_list)} comments")

    driver.quit()
    return comment_list


In [3]:
urls = ["https://www.youtube.com/watch?v=zz70o2Ia4X0", "https://www.youtube.com/watch?v=XUkV3RsjGrs","https://www.youtube.com/watch?v=UORoRpW8tnA" ,
        "https://www.youtube.com/watch?v=CM7TbmdOeAs", "https://youtu.be/zhoTX0RRXPQ", "https://youtu.be/BZPV3Cf3pLo"]

In [4]:
comments = []
for url in tqdm(urls):
    com = getComments(url)
    comments.extend(com)
    sleep(10)
    

100%|██████████| 6/6 [05:19<00:00, 53.19s/it]


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import string
import re

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers and special characters
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Remove URLs and mentions
    text = re.sub(r'http\S+', '', text)
    text = re.sub('@[^\s]+', '', text)

    # Tokenize the text into words
    words = word_tokenize(text)

    # Remove stop words
    words = [word for word in words if word not in stop_words]

    # Stem the words
    words = [stemmer.stem(word) for word in words]

    # Join the words back into a string
    text = ' '.join(words)

    return text
